# Advent of Code 2024 Day 4 

### Setup

First we need to instantiate some control variables and read in our example and testing data.

In [ ]:
from aocd import get_data, submit

day = 4
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

Both the test and sample data will be stored as a string. The string is unique to the given problem so we will need to implement parse_data in order to store it in a data structure that is useful!

In [ ]:
def parse_data(raw_data):
    return raw_data.split() # you should parse data here

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

### Part One!

In [ ]:
use_sample_data = True
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

In [ ]:
def get_col(grid:list[list[str]], i:int):
    return [ row[i] for row in grid ]

def get_cols(grid: list[list[str]]):
    cols = []
    for i in range(len(grid)):
        cols.append(get_col(grid, i))

    return cols

In [ ]:
def get_diagonal(grid: list[list[str]]) -> list[str]:
    diagonal = []
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if i == j:
                diagonal.append(grid[i][j])

    return diagonal


In [ ]:
def get_filter(grid:list[list[str]], size:int, row:int, col:int) -> list[list[str]]:
    max_row = len(grid[0])
    max_col = len(grid) 
    
    upper_bound_row = min(row + size, max_row)
    upper_bound_col = min(col + size, max_col)

    rows = grid[row : upper_bound_row]

    return [ row[col : upper_bound_col] for row in rows]


In [ ]:
def search_filter(grid_filter: list[list[str]], search_term: str):
    search_term_reverse = search_term[::-1]

    rows = ["".join(row) for row in grid_filter]
    cols = ["".join(col) for col in get_cols(grid_filter)]
    
    diag_left_to_right = "".join(get_diagonal(grid_filter))
    diag_right_to_left = "".join(get_diagonal([ row[::-1] for row in grid_filter ]))

    matches = []

    # Horizontal
    for row_idx, row in enumerate(rows):
        if search_term in row:
            col_idx = row.index(search_term)
            matches.append((row_idx, col_idx, "horizontal"))

        if search_term_reverse in row:
            col_idx = row.index(search_term_reverse)
            matches.append((row_idx, col_idx, "horizontal-reverse"))

    # Vertical
    for col_idx, col in enumerate(cols):
        if search_term in col:
            row_idx = col.index(search_term)
            matches.append((row_idx, col_idx, "vertical"))

        if search_term_reverse in col:
            row_idx = col.index(search_term_reverse)
            matches.append((row_idx, col_idx, "vertical-reverse"))

    # Diagonal left to right
    if search_term in diag_left_to_right:
        idx = diag_left_to_right.index(search_term)
        matches.append((idx, idx, "diagonal-left-to-right"))
    
    if search_term_reverse in diag_left_to_right:
        idx = diag_left_to_right.index(search_term_reverse)
        matches.append((idx, idx, "diagonal-left-to-right-reverse"))
    
    # Diagonal right to left
    if search_term in diag_right_to_left:
        idx = diag_right_to_left.index(search_term)
        matches.append((idx, len(diag_right_to_left) - idx - 1, "diagonal-right-to-left"))

    if search_term_reverse in diag_right_to_left:
        idx = diag_right_to_left.index(search_term_reverse)
        matches.append((idx, len(diag_right_to_left) - idx - 1, "diagonal-right-to-left-reverse"))

    return matches


In [ ]:
def filter_search(grid: list[list[str]], search_term: str) -> dict:
    row_size = len(grid)
    col_size = len(grid[0])
    filter_size = len(search_term)
    results = { "matches": [], "count": 0 }
    visited = set()  # To track visited positions

    for i in range(row_size - filter_size + 1):
        for j in range(col_size - filter_size + 1):
            # Get the grid filter
            grid_filter = get_filter(grid, filter_size, i, j)

            # Get matches in the local filter
            match_positions = search_filter(grid_filter, search_term)

            # Normalize positions and avoid duplicates
            for local_row, local_col, direction in match_positions:
                global_row, global_col = i + local_row, j + local_col  # Normalize indices
                global_position = (global_row, global_col, direction)

                if global_position not in visited:
                    visited.add(global_position)
                    results["matches"].append(global_position)
                    results['count'] += 1

    return results


In [ ]:
results = filter_search(data, search_term='XMAS')
results

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=results['count'], part='a', day=day, year=year, reopen=True)

### Part Two!

In [ ]:
use_sample_data = True
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data

In [ ]:
def matches_pattern(grid: list[list[str]]) -> bool:
    search_term = 'MAS'
    search_term_reverse = search_term[::-1]

    diag_left_to_right = "".join(get_diagonal(grid))
    diag_right_to_left = "".join(get_diagonal([ row[::-1] for row in grid ]))

    # check diagonal left-to-right
    if (search_term not in diag_left_to_right) and (search_term_reverse not in diag_left_to_right):
        return False
    
    if (search_term not in diag_right_to_left) and (search_term_reverse not in diag_right_to_left):
        return False
    
    return True

In [ ]:
def pattern_search(grid: list[list[str]], filter_size:int, pattern:callable) -> dict:
    row_size = len(grid)
    col_size = len(grid[0])

    result = { "matches": [], "count": 0 }

    for i in range(row_size - filter_size + 1):
        for j in range(col_size - filter_size + 1):
            grid_filter = get_filter(grid=grid, size=filter_size, row=i, col=j)

            if pattern(grid_filter):
                result["matches"].append((i, j))
                result["count"] += 1

    return result

In [ ]:
result = pattern_search(data, 3, matches_pattern)

result

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=result['count'], part='b', day=day, year=year, reopen=True)